In [ ]:
!pip install transformers datasets huggingface_hub torch --quiet


In [ ]:
import torch
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments


In [ ]:
dataset = load_dataset("imdb")

tokenizer = AutoTokenizer.from_pretrained("gpt2")

tokenizer.pad_token = tokenizer.eos_token

def tokenize_function(examples):
    model_inputs = tokenizer(examples["text"], truncation=True, padding="max_length", max_length=256)
    model_inputs["labels"] = model_inputs["input_ids"].copy()
    return model_inputs


tokenized_datasets = dataset.map(tokenize_function, batched=True)


In [ ]:
model = AutoModelForCausalLM.from_pretrained("gpt2")

training_args = TrainingArguments(
    output_dir="./gpt2-imdb-finetuned",
    eval_strategy="epoch",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    save_strategy="epoch",
    logging_dir="./logs",
    logging_steps=500,
    save_total_limit=2,
    report_to="none",
)


In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"].shuffle(seed=42).select(range(5000)),
    eval_dataset=tokenized_datasets["test"].shuffle(seed=42).select(range(1000))
)

trainer.train()


`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Epoch,Training Loss,Validation Loss
1,2.942600,2.772708
2,2.726700,2.764638
3,2.644700,2.768109


TrainOutput(global_step=3750, training_loss=2.772529459635417, metrics={'train_runtime': 1274.7379, 'train_samples_per_second': 11.767, 'train_steps_per_second': 2.942, 'total_flos': 1959690240000000.0, 'train_loss': 2.772529459635417, 'epoch': 3.0})

In [ ]:
notebook_login()

In [ ]:
model.push_to_hub("Flake56/gpt2-imdb-finetuned")
tokenizer.push_to_hub("Flake56/gpt2-imdb-finetuned")
